<a href="https://colab.research.google.com/github/divyamohan1993/cybersecurity/blob/main/Homomorphic_Encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Partially Homomorphic Voting System using Paillier Encryption**:

---

### **Introduction**

This implementation demonstrates a **Homomorphic Voting System** using the **Paillier encryption scheme**, which supports **additive homomorphism**. Additive homomorphism allows encrypted values to be added together without needing to decrypt them first. In this case, the system is designed for voting, where each vote is encrypted and later aggregated while still encrypted, preserving privacy. The results can only be decrypted by the election authority, ensuring the secrecy of individual votes.

### **Objectives**:

1. **Generate Encryption Keys**: Create public and private keys using the Paillier encryption scheme.
2. **Encrypt Votes**: Each vote is encrypted before being stored, ensuring that individual votes remain private.
3. **Aggregate Encrypted Votes**: Add the encrypted votes without decrypting them.
4. **Decrypt Results**: Only the election authority can decrypt the total votes, which are still secure during the aggregation process.
5. **Vote Integrity with Hashing**: Generate a hash for each team's encrypted votes to ensure the integrity of the voting process.

---

### **Overview of Key Concepts**:

1. **Paillier Cryptosystem**:
   - A **homomorphic encryption** scheme that supports addition on encrypted values.
   - **Public key (n, g)**: Used to encrypt votes.
   - **Private key (λ, μ)**: Used to decrypt aggregated votes.

2. **Vote Aggregation**:
   - Encrypted votes can be aggregated (added) without being decrypted, ensuring the privacy of individual votes.

3. **Hashing for Vote Integrity**:
   - The system generates a **SHA-256 hash** of each team's votes to ensure that votes haven't been tampered with during the voting process.

---


In [ ]:
# @title Demonstration Code
# Install the required package
!pip install pycryptodome

import hashlib
from Crypto.Util import number

# Function to generate keys
def generate_keys(key_size):
    p = number.getPrime(key_size)
    q = number.getPrime(key_size)
    n = p * q
    g = n + 1
    λ = (p - 1) * (q - 1)
    μ = pow(λ, -1, n)
    public_key = (n, g)
    private_key = (λ, μ)
    return public_key, private_key

# Function to encrypt a vote
def encrypt_vote(vote, public_key):
    n, g = public_key
    r = number.getRandomRange(1, n)
    c = pow(g, vote, n ** 2) * pow(r, n, n ** 2) % n ** 2
    return c

# Function to aggregate encrypted votes
def aggregate_votes(encrypted_votes, public_key):
    n, _ = public_key
    total = 1
    for c in encrypted_votes:
        total = total * c % n ** 2
    return total

# Function to decrypt the total encrypted votes
def decrypt_total(total_ciphertext, public_key, private_key):
    n, _ = public_key
    λ, μ = private_key
    x = pow(total_ciphertext, λ, n ** 2) - 1
    plaintext = ((x // n) * μ) % n
    return plaintext

# Function to calculate hash of votes for a team
def calculate_hash(encrypted_votes):
    vote_data = ''.join(str(vote) for vote in encrypted_votes)
    return hashlib.sha256(vote_data.encode()).hexdigest()

# Initialize teams and votes
teams = {'Team 1': [], 'Team 2': [], 'Team 3': [], 'Team 4': []}
team_names = list(teams.keys())

# Key generation by Election Authority
public_key, private_key = generate_keys(512)

print("Enter votes for 4 teams (0 for 'No', 1 for 'Yes'). Type 'exit' to finish voting.")

while True:
    for i, team in enumerate(team_names):
        # Ask for vote input
        user_input = input(f"Enter vote for {team} (0/1) or type 'exit' to stop: ")

        if user_input.lower() == 'exit':
            print("Voting ended.")
            break

        try:
            vote = int(user_input)
            if vote not in [0, 1]:
                print("Invalid vote. Please enter 0 or 1.")
                continue
        except ValueError:
            print("Invalid input. Please enter a number.")
            continue

        # Encrypt the vote
        encrypted_vote = encrypt_vote(vote, public_key)
        teams[team].append(encrypted_vote)

        # Display the hash for the current team's votes
        team_hash = calculate_hash(teams[team])
        print(f"Current vote hash for {team}: {team_hash}")

    # If voting has ended
    if user_input.lower() == 'exit':
        break

# Aggregating votes and decrypting the result for each team
for team, votes in teams.items():
    if votes:
        total_encrypted = aggregate_votes(votes, public_key)
        total_votes = decrypt_total(total_encrypted, public_key, private_key)
        print(f"Total 'Yes' votes for {team}: {total_votes}")
    else:
        print(f"No votes cast for {team}.")


Enter votes for 4 teams (0 for 'No', 1 for 'Yes'). Type 'exit' to finish voting.
Enter vote for Team 1 (0/1) or type 'exit' to stop: 1
Current vote hash for Team 1: 95860b1e3311a953f536cde909e2728fb7ee3b5658f73664f4a3f6f3833faf6c
Enter vote for Team 2 (0/1) or type 'exit' to stop: 0
Current vote hash for Team 2: 10426b80144ae0de00d5d86b8ffc038299a74ae420c23e9d617d6d75c0afcc46
Enter vote for Team 3 (0/1) or type 'exit' to stop: 1
Current vote hash for Team 3: 58f6acf39c99b4a28069a2c6f678fa01b57a24c8321ca350fab3dc9388619b80
Enter vote for Team 4 (0/1) or type 'exit' to stop: 1
Current vote hash for Team 4: 275a383963f8ed77108f17edf64b9f4a8dcf31ef1e5dc9b71dffb81d85bdb212
Enter vote for Team 1 (0/1) or type 'exit' to stop: 0
Current vote hash for Team 1: 0069894d6283065e710a0614215954e574f59273597704fa07d9b5292121fd1a
Enter vote for Team 2 (0/1) or type 'exit' to stop: 0
Current vote hash for Team 2: 9fb86032f53d18ca779ad07b60a959a93a44c1109c94ad46d6b48f145561a2dc
Enter vote for Team 3 (0/

---
### **Execution Steps**:

#### 1. **Install Required Library**

Before running the code, install the **PyCryptodome** library, which provides cryptographic functions used in this implementation:

```bash
!pip install pycryptodome
```

#### 2. **Generating Encryption Keys**

The election authority generates a **public** and **private key** pair using the Paillier encryption scheme. These keys will be used for encryption and decryption:

```python
from Crypto.Util import number

# Function to generate keys
def generate_keys(key_size):
    p = number.getPrime(key_size)
    q = number.getPrime(key_size)
    n = p * q
    g = n + 1
    λ = (p - 1) * (q - 1)
    μ = pow(λ, -1, n)
    public_key = (n, g)
    private_key = (λ, μ)
    return public_key, private_key

# Key generation by Election Authority
public_key, private_key = generate_keys(512)
```

- **p** and **q** are large prime numbers.
- **n** is the modulus used for both encryption and decryption.
- **g** is a base used in the encryption process.
- **λ** and **μ** are used in the decryption process.

#### 3. **Encrypting a Vote**

Each vote is encrypted using the **public key** before being stored:

```python
# Function to encrypt a vote
def encrypt_vote(vote, public_key):
    n, g = public_key
    r = number.getRandomRange(1, n)
    c = pow(g, vote, n ** 2) * pow(r, n, n ** 2) % n ** 2
    return c
```

- The vote is either `0` (No) or `1` (Yes).
- **Randomness** (r) is introduced during encryption to ensure that two identical votes will result in different ciphertexts.

#### 4. **Aggregating Encrypted Votes**

The system aggregates encrypted votes **without decryption** using homomorphic addition:

```python
# Function to aggregate encrypted votes
def aggregate_votes(encrypted_votes, public_key):
    n, _ = public_key
    total = 1
    for c in encrypted_votes:
        total = total * c % n ** 2
    return total
```

- Aggregation is done by multiplying the encrypted votes modulo \( n^2 \).
- This operation allows votes to be summed without revealing individual votes.

#### 5. **Decrypting the Total Votes**

The **private key** is used to decrypt the total number of `Yes` votes:

```python
# Function to decrypt the total encrypted votes
def decrypt_total(total_ciphertext, public_key, private_key):
    n, _ = public_key
    λ, μ = private_key
    x = pow(total_ciphertext, λ, n ** 2) - 1
    plaintext = ((x // n) * μ) % n
    return plaintext
```

- Only the election authority has access to the **private key**, which is used to decrypt the aggregated encrypted votes.

#### 6. **Calculating Hash of Encrypted Votes**

For integrity verification, a **SHA-256 hash** of encrypted votes is generated for each team:

```python
import hashlib

# Function to calculate hash of votes for a team
def calculate_hash(encrypted_votes):
    vote_data = ''.join(str(vote) for vote in encrypted_votes)
    return hashlib.sha256(vote_data.encode()).hexdigest()
```

- The hash ensures that the encrypted votes for each team have not been tampered with.

#### 7. **Interactive Voting**

The program allows users to input votes interactively for multiple teams. Votes are encrypted and aggregated:

```python
# Initialize teams and votes
teams = {'Team 1': [], 'Team 2': [], 'Team 3': [], 'Team 4': []}
team_names = list(teams.keys())

print("Enter votes for 4 teams (0 for 'No', 1 for 'Yes'). Type 'exit' to finish voting.")

while True:
    for i, team in enumerate(team_names):
        # Ask for vote input
        user_input = input(f"Enter vote for {team} (0/1) or type 'exit' to stop: ")
        
        if user_input.lower() == 'exit':
            print("Voting ended.")
            break
        
        try:
            vote = int(user_input)
            if vote not in [0, 1]:
                print("Invalid vote. Please enter 0 or 1.")
                continue
        except ValueError:
            print("Invalid input. Please enter a number.")
            continue

        # Encrypt the vote
        encrypted_vote = encrypt_vote(vote, public_key)
        teams[team].append(encrypted_vote)
        
        # Display the hash for the current team's votes
        team_hash = calculate_hash(teams[team])
        print(f"Current vote hash for {team}: {team_hash}")
    
    # If voting has ended
    if user_input.lower() == 'exit':
        break
```

---

### **Results and Decryption**

After the voting process ends, the total votes are decrypted for each team:

```python
# Aggregating votes and decrypting the result for each team
for team, votes in teams.items():
    if votes:
        total_encrypted = aggregate_votes(votes, public_key)
        total_votes = decrypt_total(total_encrypted, public_key, private_key)
        print(f"Total 'Yes' votes for {team}: {total_votes}")
    else:
        print(f"No votes cast for {team}.")
```

---

### **Conclusion**

This voting system demonstrates the power of **homomorphic encryption** using the **Paillier cryptosystem**. Votes remain encrypted throughout the process, ensuring privacy and security. Only the election authority can decrypt the aggregated total votes, preventing the exposure of individual voting preferences. Additionally, **hashing** ensures the integrity of votes, protecting against tampering during the election process.

This system can be extended to real-world scenarios where privacy and data integrity are essential, such as online voting and secure data aggregation.

# **Fully Homomorphic Encrypted Salary Calculation using TenSEAL (BFV Scheme)**

---

### **Introduction**

In this demonstration, we use **Fully Homomorphic Encryption (FHE)** to encrypt employee salary data using the **BFV scheme** provided by the **TenSEAL** library. FHE allows computations to be performed directly on encrypted data without needing to decrypt it, maintaining privacy and security throughout the process.

### **Objective**:

1. **Encrypt Salary Data**: Use the **BFV scheme** from TenSEAL to encrypt salary data for employees.
2. **Perform Operations on Encrypted Data**: We will compute the **total sum** of all encrypted salaries without decrypting them.
3. **Decrypt Results**: Finally, decrypt and display the result, verifying that operations were performed correctly on the encrypted data.

---

### **Overview of Key Concepts**:

1. **Fully Homomorphic Encryption (FHE)**:
   - Allows computations (like addition and multiplication) to be performed on encrypted data.
   - The result of the computation remains encrypted and can be decrypted later using the secret key.

2. **BFV (Brakerski-Fan-Vercauteren) Scheme**:
   - A homomorphic encryption scheme that supports addition and multiplication on encrypted integers.

3. **TenSEAL**:
   - A Python library that provides tools for **homomorphic encryption** with easy-to-use APIs for machine learning and data privacy.

---

In [ ]:
# @title Demonstration Code
!pip install tenseal

import tenseal as ts

# Create a context for encryption using BFV scheme (supports addition and multiplication)
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=8192, plain_modulus=1032193)
context.generate_galois_keys()

# Public and secret keys are generated
secret_key = context.secret_key()

# Initialize a list to store encrypted salaries
encrypted_salaries = []

# Function to encrypt a salary and display the encrypted value
def encrypt_salary(salary):
    encrypted_salary = ts.bfv_vector(context, [salary])  # Encrypt the salary as a vector
    encrypted_salaries.append(encrypted_salary)          # Store the encrypted salary
    return encrypted_salary

# Function to calculate the total of encrypted salaries
def calculate_total():
    if len(encrypted_salaries) == 0:
        print("No salaries entered yet.")
        return None
    total_encrypted = encrypted_salaries[0]
    for encrypted_salary in encrypted_salaries[1:]:
        total_encrypted += encrypted_salary
    return total_encrypted

# Function to decrypt and display the total salary
def decrypt_total(total_encrypted):
    total_decrypted = total_encrypted.decrypt(secret_key)
    print(f"\nTotal Salary Decrypted: {total_decrypted[0]}")  # Decrypted total

# Main program to enter salaries and show the process
print("Enter employee salaries (type 'done' to finish):")

while True:
    user_input = input("Enter salary or type 'done': ")

    if user_input.lower() == 'done':
        print("\nFinalizing salary encryption and calculation...\n")
        break

    try:
        salary = int(user_input)
        encrypted_salary = encrypt_salary(salary)  # Encrypt the salary

        # Show the encrypted salary (prove it's encrypted)
        print(f"Encrypted salary: {encrypted_salary.serialize()[:100]}... (truncated)")  # Truncated for display

        # Calculate and show the encrypted total as we proceed
        total_encrypted = calculate_total()
        if total_encrypted:
            print(f"Encrypted total after this salary: {total_encrypted.serialize()[:100]}... (truncated)\n")

    except ValueError:
        print("Please enter a valid number or 'done' to finish.")

# Decrypt and display the final total
if total_encrypted:
    decrypt_total(total_encrypted)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 54.5 MB/s eta 0:00:00
Enter employee salaries (type 'done' to finish):
Enter salary or type 'done': 1000
Encrypted salary: b'\n\x01\x01\x12\xbe\xb1\x1a^\xa1\x10\x04\x01\x02\x00\x00\xbe\x98\x06\x00\x00\x00\x00\x00(\xb5/\xfd\xa0a\x00\x08\x00\x0c\x16\rn\xfd_\xabh)\x10\xb0\x1d\xe6\xaf\x15\xa0EC\xd8\x1f\xcf\xc2\x9f(\xd2\xca\x08\x8e\xb3b\x9e\xef\xa9\x99\xff\xd7\xca\x97m_v\x9f\xa2\x15AJ)\xa5\x94)\xa2h\x92h\xabh\xb3)\x16I2\x92\xf3+\x92\xc5\xc5K'... (truncated)
Encrypted total after this salary: b'\n\x01\x01\x12\xbe\xb1\x1a^\xa1\x10\x04\x01\x02\x00\x00\xbe\x98\x06\x00\x00\x00\x00\x00(\xb5/\xfd\xa0a\x00\x08\x00\x0c\x16\rn\xfd_\xabh)\x10\xb0\x1d\xe6\xaf\x15\xa0EC\xd8\x1f\xcf\xc2\x9f(\xd2\xca\x08\x8e\xb3b\x9e\xef\xa9\x99\xff\xd7\xca\x97m_v\x9f\xa2\x15AJ)\xa5\x94)\xa2h\x92h\xabh\xb3)\x16I2\x92\xf3+\x92\xc5\xc5K'... (truncated)

Enter salary or type 'done': 9902
Encrypted salary: b'\n\x01\x01\x12\xc2\xb2\x1a^\xa1\x10\x04\x01\x02\x00\x00B\x99\x06

---
### **Execution Steps**

#### 1. **Install TenSEAL**

To use the TenSEAL library, it must first be installed. This can be done via:

```bash
!pip install tenseal
```

#### 2. **Setting Up the Encryption Context**

The BFV scheme requires a context to be set up that holds encryption parameters like polynomial degree and plaintext modulus:

```python
import tenseal as ts

# Create a context for the BFV scheme (supports addition and multiplication)
context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=8192, plain_modulus=1032193)

# Generate Galois keys for encrypted vector operations
context.generate_galois_keys()

# Public and secret keys are generated as part of the context
secret_key = context.secret_key()
```

- **poly_modulus_degree**: Defines the degree of the polynomial, affecting the size of ciphertexts and security.
- **plain_modulus**: A large prime number that determines the size of plaintext space.

#### 3. **Encrypting Salaries**

We encrypt each salary using the BFV scheme. Salaries are encrypted as vectors and stored for further operations:

```python
# Initialize a list to store encrypted salaries
encrypted_salaries = []

# Function to encrypt a salary
def encrypt_salary(salary):
    encrypted_salary = ts.bfv_vector(context, [salary])  # Encrypt the salary as a vector
    encrypted_salaries.append(encrypted_salary)          # Store the encrypted salary
    return encrypted_salary
```

#### 4. **Calculating the Encrypted Total**

Without decrypting the salaries, we can calculate the total of all encrypted salaries. The result remains encrypted:

```python
# Function to calculate the total of encrypted salaries
def calculate_total():
    if len(encrypted_salaries) == 0:
        print("No salaries entered yet.")
        return None
    total_encrypted = encrypted_salaries[0]
    for encrypted_salary in encrypted_salaries[1:]:
        total_encrypted += encrypted_salary  # Addition performed on encrypted values
    return total_encrypted
```

#### 5. **Decrypting and Displaying the Total**

Finally, we decrypt the total and display the decrypted result. The decryption uses the **secret key**:

```python
# Function to decrypt and display the total salary
def decrypt_total(total_encrypted):
    total_decrypted = total_encrypted.decrypt(secret_key)  # Decrypt using the secret key
    print(f"\nTotal Salary Decrypted: {total_decrypted[0]}")  # Decrypted total
```

#### 6. **Interactive Salary Input and Final Total Decryption**

The program interactively accepts salaries, encrypts them, shows the encrypted values, and calculates the encrypted total as you proceed:

```python
# Main program to enter salaries and show the process
print("Enter employee salaries (type 'done' to finish):")

while True:
    user_input = input("Enter salary or type 'done': ")

    if user_input.lower() == 'done':
        print("\nFinalizing salary encryption and calculation...\n")
        break

    try:
        salary = int(user_input)
        encrypted_salary = encrypt_salary(salary)  # Encrypt the salary
        
        # Show the encrypted salary (proving it's encrypted)
        print(f"Encrypted salary: {encrypted_salary.serialize()[:100]}... (truncated)")  # Truncated for display
        
        # Calculate and show the encrypted total
        total_encrypted = calculate_total()
        if total_encrypted:
            print(f"Encrypted total after this salary: {total_encrypted.serialize()[:100]}... (truncated)\n")
        
    except ValueError:
        print("Please enter a valid number or 'done' to finish.")

# Decrypt and display the final total
if total_encrypted:
    decrypt_total(total_encrypted)
```

---

### **Summary of Functionality**:

1. **Encrypting Salaries**: Each salary is encrypted using BFV and stored in a list.
2. **Performing Operations on Encrypted Data**: Salaries are added together while still encrypted.
3. **Decrypting the Result**: The final total is decrypted and displayed, ensuring that the operations on encrypted data were correct.

---

### **Conclusion**

This example demonstrates how to securely encrypt and perform operations on sensitive data using **Fully Homomorphic Encryption (FHE)** with the **BFV scheme** provided by **TenSEAL**. It showcases the capability to manipulate encrypted data and decrypt results only when necessary, ensuring privacy and security of employee salary data.



# **Simulation of Fully Homomorphic Encryption (FHE) Using Caesar Cipher**

### **Introduction:**
Fully Homomorphic Encryption (FHE) allows computations to be performed on encrypted data, ensuring privacy and security throughout the process. FHE guarantees that even without decrypting the data, the operations can be carried out, and the result can later be decrypted to obtain the correct outcome.

This notebook demonstrates a basic FHE concept using a **Caesar Cipher**, where employee salaries are encrypted, manipulated (incremented, taxed, and bonuses added), and stored in a file. The operations are performed on the **encrypted values**, and the results are encrypted back, maintaining data privacy throughout.

### **Objective:**
- Encrypt the employee's salary using Caesar Cipher (basic FHE simulation).
- Store encrypted salaries in a file for secure retrieval.
- Perform operations (increment, tax deduction, bonus addition) on the encrypted data.
- Ensure that data is only decrypted when needed and re-encrypted after every operation.
- Verify and validate the correct working of FHE principles by operating on encrypted data.

---

### **Caesar Cipher Encryption and Decryption**

The Caesar Cipher is a basic encryption technique that shifts the value by a fixed amount (`key`). While it is not suitable for actual encryption in production systems, it helps demonstrate the concept of operating on encrypted data. Here's how the Caesar Cipher works in this simulation:

- **Encryption**: The salary is shifted by a key (e.g., `key = 3`) to encrypt the value.
- **Decryption**: The encrypted value is shifted back by the same key to retrieve the original salary.

### **Key Steps of FHE Validation:**
1. **Encrypt Salary**: When a new salary is input, it is encrypted using the Caesar Cipher and stored.
2. **Fetch and Decrypt Salary**: When an existing employee ID is entered, the encrypted salary is retrieved, decrypted, and displayed for verification.
3. **Perform Operations on Encrypted Data**: Operations like increment, tax deduction, and bonus addition are performed on the decrypted salary, and the results are encrypted after each operation.
4. **Store Updated Encrypted Salary**: After each operation, the updated encrypted salary is stored in the file.

---

In [ ]:
# @title Demonstration Code
import json

# Simple Caesar Cipher functions for simulation of FHE
def caesar_encrypt(number, key):
    """Encrypt a number using Caesar Cipher by shifting it by key."""
    return number + key

def caesar_decrypt(number, key):
    """Decrypt a number using Caesar Cipher by reversing the shift by key."""
    return number - key

# FHE key for Caesar Cipher
key = 378356876235  # Using a fixed Caesar Cipher key of 3

# Function to apply percentage increment
def apply_increment(salary, increment_percent):
    increment_factor = (1 + (increment_percent / 100))
    return int(salary * increment_factor)

# Function to deduct tax
def deduct_tax(salary, tax_percent):
    tax_factor = (1 - (tax_percent / 100))
    return int(salary * tax_factor)

# Function to add bonus
def add_bonus(salary, bonus_amount):
    return salary + bonus_amount

# File to store employee data
employee_file = "employee_salaries.json"

# Load employee salaries from file
def load_employee_data():
    try:
        with open(employee_file, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {}

# Save employee salaries to file
def save_employee_data(data):
    with open(employee_file, "w") as file:
        json.dump(data, file)

# Main demonstration
def main():
    # Load existing employee salary data
    employee_data = load_employee_data()

    print("Enter employee ID and salaries (type 'done' to finish):")

    while True:
        emp_id = input("Enter employee ID or type 'done': ")

        if emp_id.lower() == 'done':
            print("\nFinalizing salary encryption and calculation...\n")
            break

        if emp_id in employee_data:
            # Fetch encrypted salary for the given employee ID
            encrypted_salary = employee_data[emp_id]
            decrypted_salary = caesar_decrypt(encrypted_salary, key)
            print(f"Fetched encrypted salary for employee {emp_id}: {encrypted_salary} (Decrypted: {decrypted_salary})")
        else:
            try:
                # New employee entry
                salary = int(input("Enter salary: "))
                # Encrypt the salary
                encrypted_salary = caesar_encrypt(salary, key)
                employee_data[emp_id] = encrypted_salary
                print(f"Encrypted salary for employee {emp_id}: {encrypted_salary}")
                save_employee_data(employee_data)
                decrypted_salary = salary  # Use original input for the next steps
            except ValueError:
                print("Invalid input. Please enter a valid salary.")

        try:
            # Perform increment operation
            increment_percent = float(input(f"Enter increment percentage for salary {decrypted_salary}: "))
            salary_with_increment = apply_increment(decrypted_salary, increment_percent)
            encrypted_salary = caesar_encrypt(salary_with_increment, key)  # Encrypt after increment
            employee_data[emp_id] = encrypted_salary
            save_employee_data(employee_data)
            print(f"Encrypted Salary after {increment_percent}% increment: {encrypted_salary}")

            # Perform tax deduction operation
            decrypted_salary = caesar_decrypt(employee_data[emp_id], key)  # Decrypt before tax deduction
            tax_percent = float(input(f"Enter tax percentage to deduct from incremented salary: "))
            salary_after_tax = deduct_tax(decrypted_salary, tax_percent)
            encrypted_salary = caesar_encrypt(salary_after_tax, key)  # Encrypt after tax deduction
            employee_data[emp_id] = encrypted_salary
            save_employee_data(employee_data)
            print(f"Encrypted Salary after {tax_percent}% tax deduction: {encrypted_salary}")

            # Perform bonus addition operation
            decrypted_salary = caesar_decrypt(employee_data[emp_id], key)  # Decrypt before bonus
            bonus_amount = float(input(f"Enter bonus amount to add to salary after tax: "))
            salary_with_bonus = add_bonus(decrypted_salary, bonus_amount)
            encrypted_salary = caesar_encrypt(salary_with_bonus, key)  # Encrypt after bonus addition
            employee_data[emp_id] = encrypted_salary
            save_employee_data(employee_data)
            print(f"Encrypted Salary after adding {bonus_amount} bonus: {encrypted_salary}\n")

        except ValueError:
            print("Invalid input. Please enter valid numbers.")

if __name__ == "__main__":
    main()


Enter employee ID and salaries (type 'done' to finish):
Enter employee ID or type 'done': 1
Fetched encrypted salary for employee 1: 423548081147.0 (Decrypted: 45191204912.0)
Enter increment percentage for salary 45191204912.0: -10
Encrypted Salary after -10.0% increment: 419028960655
Enter tax percentage to deduct from incremented salary: 33443
Encrypted Salary after 33443.0% tax deduction: -13182936231925
Enter bonus amount to add to salary after tax: 11
Encrypted Salary after adding 11.0 bonus: -13182936231914.0

Enter employee ID or type 'done': done

Finalizing salary encryption and calculation...



### **Key Features**

1. **Caesar Cipher Simulation**:
   - Basic encryption (shift by `key = 3`) and decryption functions simulate the concept of working on encrypted data.
   - **Encrypted data** is stored and retrieved from a file (`employee_salaries.json`).

2. **File-Based Storage**:
   - Encrypted salaries are stored persistently in a JSON file. Each employee's salary is mapped to their employee ID.
   - This helps simulate a real-world scenario of encrypted salary storage and retrieval.

3. **Operations on Encrypted Data**:
   - **Increment**, **tax deduction**, and **bonus addition** are performed on decrypted data.
   - After each operation, the result is encrypted again and stored, maintaining the principle of working on encrypted data.

4. **Secure Data Handling**:
   - Even though operations are performed on the decrypted values, the entire workflow simulates FHE by ensuring that all results are immediately re-encrypted.
   - The actual salary values are not exposed outside the scope of the program’s logic.

---

### **Conclusion:**
This code demonstrates the fundamental principles of **Fully Homomorphic Encryption (FHE)** by simulating the process with **Caesar Cipher**. While Caesar Cipher is used here for simplicity, the workflow validates the concept of manipulating encrypted data without revealing it, as required in actual FHE systems. This demonstration is suitable for educational purposes and is an effective way to explain the concept of FHE to faculty and peers.

# **Fully Homomorphic-Like Encryption for Salary Data Using AES**

### **Introduction**

This demonstration showcases how employee salary data is encrypted, stored, and retrieved securely using **AES encryption** in CBC mode. The goal is to simulate the idea of performing operations on encrypted data and ensure that the data can persist across multiple program sessions (i.e., program can be stopped and restarted without data loss or corruption).

### **Objectives:**

1. **Encrypt Employee Salaries**: We will encrypt the salary data using **AES (Advanced Encryption Standard)**.
2. **Perform Operations on Encrypted Data**: Salary increment, tax deduction, and bonus addition are performed, and the results are encrypted back.
3. **Key and Data Persistence**: The AES key and encrypted employee data will be stored in files (`aes_key.key` and `employee_salaries_aes.json`) to allow for multiple program sessions.
4. **Secure Data Handling**: The data remains encrypted both in transit and at rest. Only authorized operations decrypt the salary temporarily.

### **AES Encryption Overview:**
- **AES (Advanced Encryption Standard)** is a secure and widely used symmetric encryption algorithm.
- We use **AES in CBC mode** (Cipher Block Chaining) to ensure that the data is encrypted securely with an initialization vector (IV) for each encryption.
- We apply **PKCS7 padding** to ensure the salary fits into AES’s block size.

---

### **Key Features**:

1. **Key Management**:
   - A **256-bit AES key** is generated and saved in a file (`aes_key.key`) for reuse across multiple sessions.
   - The same key is used for encryption and decryption, ensuring data consistency across restarts.

2. **Employee Data Storage**:
   - Encrypted salaries are stored in a file (`employee_salaries_aes.json`) in **hexadecimal format** for easy serialization and persistence.
   - The encrypted data is reloaded from the file on subsequent program executions.

3. **Operations**:
   - The program allows three operations to be performed on the decrypted salary: **increment**, **tax deduction**, and **bonus addition**.
   - After each operation, the updated salary is encrypted and stored securely.

---


In [ ]:
# @title Demonstration Code

import os
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend

# File paths
key_file = "aes_key.key"
employee_file = "employee_salaries_aes.json"

# AES block size and IV size
BLOCK_SIZE = 128  # AES block size (128 bits)
IV_SIZE = 16  # AES IV size (16 bytes)

# Padding and unpadding functions for AES (ensures data fits into block size)
def pad_data(data):
    padder = padding.PKCS7(BLOCK_SIZE).padder()
    padded_data = padder.update(data) + padder.finalize()
    return padded_data

def unpad_data(padded_data):
    unpadder = padding.PKCS7(BLOCK_SIZE).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()
    return data

# AES encryption function
def aes_encrypt(data, key):
    iv = os.urandom(IV_SIZE)  # Generate a new IV for each encryption
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Pad the data to fit into block size
    padded_data = pad_data(data)

    # Encrypt the padded data
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
    return iv + encrypted_data  # Prepend IV to encrypted data

# AES decryption function
def aes_decrypt(encrypted_data, key):
    iv = encrypted_data[:IV_SIZE]  # Extract IV from the encrypted data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the data
    padded_data = decryptor.update(encrypted_data[IV_SIZE:]) + decryptor.finalize()

    # Unpad the data to retrieve the original
    return unpad_data(padded_data)

# Convert number (float or int) to bytes for encryption
def number_to_bytes(number):
    return str(number).encode('utf-8')

# Convert bytes back to a floating-point number after decryption
def bytes_to_number(byte_data):
    return float(byte_data.decode('utf-8'))

# Functions to apply operations on salary
def apply_increment(salary, increment_percent):
    increment_factor = (1 + (increment_percent / 100))
    return salary * increment_factor

def deduct_tax(salary, tax_percent):
    tax_factor = (1 - (tax_percent / 100))
    return salary * tax_factor

def add_bonus(salary, bonus_amount):
    return salary + bonus_amount

# Save AES key to file
def save_aes_key(key):
    with open(key_file, "wb") as file:
        file.write(key)

# Load AES key from file
def load_aes_key():
    with open(key_file, "rb") as file:
        return file.read()

# Generate a new AES key if one doesn't exist, otherwise load the existing key
def initialize_aes_key():
    if os.path.exists(key_file):
        print("Loading existing AES key...")
        return load_aes_key()
    else:
        print("Generating new AES key...")
        key = os.urandom(32)  # Generate a 256-bit AES key
        save_aes_key(key)
        return key

# Load employee salaries from file
def load_employee_data():
    try:
        with open(employee_file, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        return {}

# Save employee salaries to file
def save_employee_data(data):
    with open(employee_file, "w") as file:
        json.dump(data, file)

# Main demonstration
def main():
    # Load or initialize the AES key
    aes_key = initialize_aes_key()

    # Load existing employee salary data
    employee_data = load_employee_data()

    print("Enter employee ID and salaries (type 'done' to finish):")

    while True:
        emp_id = input("Enter employee ID or type 'done': ")

        if emp_id.lower() == 'done':
            print("\nDone\n")
            break

        if emp_id in employee_data:
            # Fetch encrypted salary for the given employee ID
            encrypted_salary = bytes.fromhex(employee_data[emp_id])  # Convert stored hex back to bytes
            try:
                decrypted_salary = aes_decrypt(encrypted_salary, aes_key)
                decrypted_salary_num = bytes_to_number(decrypted_salary)
                print(f"Fetched encrypted salary for employee {emp_id}: (Decrypted: {decrypted_salary_num})")
            except ValueError as e:
                print(f"Error during decryption: {e}")
                continue
        else:
            try:
                # New employee entry
                salary = float(input("Enter salary: "))
                # Convert salary to bytes and encrypt it
                encrypted_salary = aes_encrypt(number_to_bytes(salary), aes_key)
                employee_data[emp_id] = encrypted_salary.hex()  # Store encrypted salary as hex string
                print(f"Encrypted salary for employee {emp_id}: (stored securely)")
                save_employee_data(employee_data)
                decrypted_salary_num = salary  # Use original input for the next steps
            except ValueError:
                print("Invalid input. Please enter a valid salary.")

        try:
            # Perform increment operation
            increment_percent = float(input(f"Enter increment percentage for salary {decrypted_salary_num}: "))
            salary_with_increment = apply_increment(decrypted_salary_num, increment_percent)
            encrypted_salary = aes_encrypt(number_to_bytes(salary_with_increment), aes_key)  # Encrypt after increment
            employee_data[emp_id] = encrypted_salary.hex()
            save_employee_data(employee_data)
            print(f"Encrypted Salary after {increment_percent}% increment: (stored securely)")

            # Perform tax deduction operation
            decrypted_salary = aes_decrypt(bytes.fromhex(employee_data[emp_id]), aes_key)  # Decrypt before tax deduction
            decrypted_salary_num = bytes_to_number(decrypted_salary)
            tax_percent = float(input(f"Enter tax percentage to deduct from incremented salary: "))
            salary_after_tax = deduct_tax(decrypted_salary_num, tax_percent)
            encrypted_salary = aes_encrypt(number_to_bytes(salary_after_tax), aes_key)  # Encrypt after tax deduction
            employee_data[emp_id] = encrypted_salary.hex()
            save_employee_data(employee_data)
            print(f"Encrypted Salary after {tax_percent}% tax deduction: (stored securely)")

            # Perform bonus addition operation
            decrypted_salary = aes_decrypt(bytes.fromhex(employee_data[emp_id]), aes_key)  # Decrypt before bonus
            decrypted_salary_num = bytes_to_number(decrypted_salary)
            bonus_amount = float(input(f"Enter bonus amount to add to salary after tax: "))
            salary_with_bonus = add_bonus(decrypted_salary_num, bonus_amount)
            encrypted_salary = aes_encrypt(number_to_bytes(salary_with_bonus), aes_key)  # Encrypt after bonus addition
            employee_data[emp_id] = encrypted_salary.hex()
            save_employee_data(employee_data)
            print(f"Encrypted Salary after adding {bonus_amount} bonus: (stored securely)\n")

            decrypted_salary = aes_decrypt(bytes.fromhex(employee_data[emp_id]), aes_key)  # Decrypt before bonus
            decrypted_salary_num = bytes_to_number(decrypted_salary)
            print("\nFinalizing salary encryption and calculation...\n")
            print(f"Salary for employee {emp_id} is {decrypted_salary_num})")

        except ValueError:
            print("Invalid input. Please enter valid numbers.")

if __name__ == "__main__":
    main()


Loading existing AES key...
Enter employee ID and salaries (type 'done' to finish):
Enter employee ID or type 'done': 1
Fetched encrypted salary for employee 1: (Decrypted: 1331117.0476560004)
Enter increment percentage for salary 1331117.0476560004: 2
Encrypted Salary after 2.0% increment: (stored securely)
Enter tax percentage to deduct from incremented salary: 7999
Encrypted Salary after 7999.0% tax deduction: (stored securely)
Enter bonus amount to add to salary after tax: 
Invalid input. Please enter valid numbers.
Enter employee ID or type 'done': 1
Fetched encrypted salary for employee 1: (Decrypted: -107247834.30623442)
Enter increment percentage for salary -107247834.30623442: 2288
Encrypted Salary after 2288.0% increment: (stored securely)
Enter tax percentage to deduct from incremented salary: 123
Encrypted Salary after 123.0% tax deduction: (stored securely)
Enter bonus amount to add to salary after tax: 1999873
Encrypted Salary after adding 1999873.0 bonus: (stored securel

---

### **Execution Instructions**

1. **First Run**:
   - The program will **generate a new AES key** and save it in `aes_key.key`. This key will be used to encrypt and decrypt employee salary data.
   - Employee salaries are entered, encrypted, and saved in `employee_salaries_aes.json` in hexadecimal format.

2. **Subsequent Runs**:
   - The program will load the saved AES key and employee salary data from their respective files.
   - The same key will be used to decrypt the salary data, allowing for seamless continuation of salary operations.

3. **Perform Operations**:
   - You can perform operations like increment, tax deduction, and bonus addition on decrypted salary data. After each operation, the updated salary is encrypted again and saved.

### **Conclusion**:
This demonstration highlights the concepts of **encryption, decryption**, and **data persistence** using AES. The program securely handles employee salary data, ensuring that encrypted information can be manipulated and persisted across multiple sessions. This is a practical demonstration of secure encryption techniques in action, applicable to real-world scenarios like payroll systems.